In [1]:
import jax
import optax
import jax.numpy as jnp
import jax.random as random
from functools import partial
import random as rand
import time

gpu_device = jax.devices('gpu')[0]
cpu_device = jax.devices('cpu')[0]

In [2]:
debug_log = print
debug_log = lambda *x: 0

In [3]:
# params
sequence_length = 2

experiment:
get my posts
input: nothing
output: a post

experiment:
video generator
input: blank transparent canvas
output: the next frame of the video
do a several convolution skip connections on each step (adds changes to each frame)

In [4]:
# process tweet data
import json

# Load the tweets.js file
with open('./tweets.js', 'r', encoding='utf-8') as file:
    # Skip the JavaScript assignment and load only the JSON part
    content = file.read()
    json_data = content.split('=', 1)[1].strip()  # Extract the JSON part after `=`
    json_data = json_data.rstrip(';')  # Remove trailing semicolon if present
    tweets_data = json.loads(json_data)

# Initialize lists for replies and posts
replies = []
posts = []

# Process each tweet in the dataset
for tweet_obj in tweets_data:
    tweet = tweet_obj["tweet"]
    
    if "in_reply_to_status_id_str" in tweet and tweet["in_reply_to_status_id_str"]:
        # It's a reply
        replies.append({
            "id": tweet["id_str"],
            "text": tweet["full_text"],
            "in_reply_to": tweet["in_reply_to_status_id_str"],
            "user": tweet.get("in_reply_to_screen_name", None),
            "created_at": tweet["created_at"]
        })
    else:
        # It's a standalone post
        posts.append({
            "id": tweet["id_str"],
            "text": tweet["full_text"],
            "created_at": tweet["created_at"]
        })

# Output the results
print("Replies:")
for reply in replies[:4]:
    print(reply["text"])

print("\nPosts:")
for post in posts[:4]:
    print(post["text"])


Replies:
@VictorTaelin ive found sonnet 3.5v2 to be surprisingly good at coding. upgraded my tools from v1 to v2 and all of a sudden i have to reprompt it like 1/3rd the time
@dgant &lt;script src="gifeditor.mp3" type="application/json"&gt;
@bozo10n 💪
@sunsettler experimentation games are the best

Posts:
compression always feels so satisfying https://t.co/mM5acJxydL
it only takes one line of code to make a gifboard btw https://t.co/hFlPyNTvRm
RT @calbch: @kuberdenis entrepreneurship is the ultimative vehicle for personal development
just two idiots playing a game of chess https://t.co/USjWySv3W9


In [31]:
# set up and process data
post_limit = 100

stop_char = "🛑"
all_posts = [
  "reply: " + reply["text"] + stop_char for reply in replies[:post_limit]
]
all_posts.extend(
  [
    "post: " + post["text"] + stop_char for post in posts[:post_limit]
  ]
)
rand.shuffle(all_posts)
print('gathered initial posts')

token_type = "word"
if token_type == "char":
  chars = "".join(all_posts)
  vocab = sorted(list(set(chars)))
  vocab_size = len(vocab)
  print("vocab size:", vocab_size)

  translations = {
    "encode" : dict([(c, t) for c, t in zip(vocab, range(len(vocab)))]),
    "decode" : dict([(t, c) for c, t in zip(vocab, range(len(vocab)))])
  }

  token = lambda c: translations["encode"][c] # char to token
  char = lambda t: translations["decode"][int(t)] # token to char
  encode = lambda cs: jnp.array([token(c) for c in cs])
  decode = lambda ts: "".join([char(t) for t in ts])

elif token_type == "word":
  text = " ".join(all_posts)
  words = text.split(' ')
  vocab = sorted(list(set(words)))
  vocab_size = len(vocab)
  print("vocab size:", vocab_size)

  translations = {
    "encode" : dict([(w, t) for w, t in zip(vocab, range(len(vocab)))]),
    "decode" : dict([(t, w) for w, t in zip(vocab, range(len(vocab)))])
  }

  token = lambda w: translations["encode"][w] # word to token
  word = lambda t: translations["decode"][int(t)] # token => word
  encode = lambda cs: jnp.array([token(w) for w in cs.split(" ")])
  decode = lambda ts: " ".join([word(t) for t in ts])


post_token_sets = [encode(post) for post in all_posts]
print('converted posts to tokens')

from tqdm import tqdm
sequence_token_sets = []
for post in tqdm(post_token_sets):
  for i in range(max(0, len(post) - sequence_length + 1)):
    sequence_token_sets.append(post[i:i+sequence_length])


print(f"loaded {len(sequence_token_sets)} sequences of len {sequence_length}.")


gathered initial posts
vocab size: 1978
converted posts to tokens


100%|██████████| 200/200 [00:00<00:00, 208.02it/s]

loaded 4274 sequences of len 2.


In [40]:
# train functions
# init rnn params
# inputs ()
def init_rnn_params(key, input_shape, hidden_shape, conveyor_shape, output_shape):
  keys = random.split(key, 4*sequence_length)
  hx_concat_length = hidden_shape[0] + input_shape[0] # todo incorporate batching
  conveyor_shape = conveyor_shape[0]
  output_size = output_shape[0]
  rnn_params = [
    # xW means shape of W is (input, output)
    # hW and xW should output the shape of h.
    {
      "Whxc" : random.normal(keys[3*n + 0], shape=(hx_concat_length, output_size)) * jnp.sqrt(2 / hx_concat_length),
      "Bhxc" : jnp.zeros(shape=(output_size,)),
      "Whxi" : random.normal(keys[3*n + 1], shape=(hx_concat_length, output_size)) * jnp.sqrt(2 / hx_concat_length),
      "Bhxi" : jnp.zeros(shape=(output_size,)),
      "Whxcupd" : random.normal(keys[3*n + 2], shape=(hx_concat_length, output_size)) * jnp.sqrt(2 / hx_concat_length),
      "Bhxcupd" : jnp.zeros(shape=(output_size,)),
      "Whxo" : random.normal(keys[3*n + 3], shape=(hx_concat_length, output_size)) * jnp.sqrt(2 / hx_concat_length),
      "Bhxo" : jnp.zeros(shape=(output_size,)),
    }
    for n in range(sequence_length + 1) # +1 for the empty sequence at the start
  ]
  return rnn_params

@partial(jax.jit, device=gpu_device)
def step_start(rnn_params, x):
  # todo incorporate batching
  hx_concat_length, conveyor_shape = rnn_params[0]["Whxc"].shape
  # x is B, C (T == 1 so its not in here)
  batch_size = x.shape[0]
  input_shape = x.shape[1]
  h_shape = hx_concat_length - input_shape
  h = jnp.zeros((batch_size, h_shape)) # batch size 1 for now
  c = jnp.zeros((batch_size, conveyor_shape))
  y, c, h = step(rnn_params, c, h, x, n=0)
  debug_log('y: ', y.shape, f"should be {('Batch', 'C(logits=vocab_size)')}")
  return y, c, h


@partial(jax.jit, static_argnames=["n"], device=gpu_device) # n is static (i think calculated at jit comptime each time a new val is passed)
def step(rnn_params, c, h, x, n):
  # dim=1 so that for batching when its (n, h+x) it concats h+x
  hxconcat = jax.lax.concatenate([h, x], dimension=1) # (batch, h+x)

  debug_log('x: ', x.shape, f"should be {('Batch', 'C(one hot encodings)')}")

  # input gate
  # (B, h+x) @ (h+x,c) => (B, c) (B = batch dimension, right now its just 1)
  wxc = rnn_params[n]["Whxc"]
  xc = hxconcat @ wxc
  b = rnn_params[n]["Bhxc"]
  f = jax.nn.sigmoid(xc + b)
  c = c * f # (B, c) * (B, c) => (B, c)

  # update gate
  # (B, h+x) @ (h+x, c) => (B, c)
  i = jax.nn.sigmoid(hxconcat @ rnn_params[n]["Whxi"] + rnn_params[n]["Bhxi"])
  # (B, h+x) @ (h+x, c) => (B, c)
  c_upd = jax.nn.tanh(hxconcat @ rnn_params[n]["Whxcupd"] + rnn_params[n]["Bhxcupd"])
  c = c + i*c_upd # (B, c) + (B, c) * (B, c) => (B, c)

  # output gate
  o = jax.nn.sigmoid(hxconcat @ rnn_params[n]["Whxo"] + rnn_params[n]["Bhxo"])
  h = o * jax.nn.tanh(c) # delete gate

  y = o
  return y, c, h

@partial(jax.jit, device=gpu_device)
def forward(rnn_params, xbow):
  ys = []
  yi, c, h = step_start(rnn_params, xbow[:, 0]) # try to predict the first token of each x in the batch
  ys.append(yi)
  for n in range(1, xbow.shape[1]):
    y, c, h = step(rnn_params, c, h, xbow[:, n], n=n) # try to predict i+1th token for each batch
    ys.append(y)
  
  ys_out = jnp.transpose(jnp.array(ys), (1, 0, 2)) # turn it from (T, B, C) to (B, T, C)
  debug_log("ys", ys_out.shape, f"should be {('batches', 3, 'C(logits=vocab_size)')}")

  return ys_out


# learned embeddings
embedding_type = "learned"
if embedding_type == "learned":
  @jax.jit
  def embed_tokens(embedding_params, tokens):
    # ts[:, None] turns it from [t, t, t, t] to [[t], [t], [t], [t]]. as it should be, a row vector. transpose but for 1d vec.
    # x = one-hot(x) -> embed(x)
    oh = jax.nn.one_hot(tokens[:, None], vocab_size, axis=1) # T, 1 => T, vocab_size
    debug_log(oh.shape, embedding_params["layer_1"]["w"].shape)
    x = oh @ embedding_params["layer_1"]["w"] + embedding_params["layer_1"]["b"] # (T, vocab_size) # (vocab_size, model_dim) => (T, model_dim)
    return x
  
  def init_embedding_params(key, vocab_size, model_dim):
    keys = random.split(key, 10)
    embedding_params = {
      "layer_1" : {
        "w" : random.normal(keys[0], shape=(vocab_size, model_dim)), # T, vocab_size => T, model_dim
        "b" : jnp.zeros((model_dim,)),
        }
    }
    return embedding_params
elif embedding_type == "one-hot":
  # one hot embeddings
  # EXPENSIVE when vocab size is high
  @partial(jax.jit, device=gpu_device)
  def embed_tokens(tokens):
    return jax.nn.one_hot(tokens, vocab_size, axis=-1)


def embed_chars(chars):
  tokens = encode(chars)
  return embed_tokens(tokens)


@partial(jax.jit, device=gpu_device)
def get_loss(rnn_params, xtokens, ytokens):
  xbow = embed_tokens(rnn_params[-1], xtokens)
  debug_log("embeddings: ", xbow.shape, "should be", "(B, T, C(one hot)) =", ("b", 3, "vocab_size"))
  logits = forward(rnn_params, xbow)
  debug_log("logits:", logits.shape, "should be", "(B, T, C) =", ("b", 3, "vocab_size"))
  vocab_size = logits.shape[-1] # channel dimension
  ytokens_one_hot = jax.nn.one_hot(ytokens, vocab_size, axis=-1)
  debug_log("ytokens_one_hot", ytokens_one_hot.shape, "should be", "(B, T, C(one hot)) =", ("b", 3, "vocab_size"))
  cross_entropies = -jnp.sum(jax.nn.log_softmax(logits, axis=-1) * ytokens_one_hot, axis=-1) # axis=-1 is along C in a (B,T,C)
  debug_log("cross_entropies", cross_entropies.shape, "should be", "(B, T, 1) =", ("b", 3, 1))
  net_cross_entropy_loss = jnp.mean(cross_entropies)
  return net_cross_entropy_loss



In [47]:
from tokenizers import CharBPETokenizer

# Your text
text = "ummmmmmmmmmm"



# Display results
print("Token IDs:", output.ids)
print("Tokens:", output.tokens)





Token IDs: [2, 5, 5, 4, 3]
Tokens: ['u', 'mmmm', 'mmmm', 'mm', 'm</w>']


In [42]:
# train
keys = random.split(random.PRNGKey(198123), 10)

if embedding_type == "learned":
  model_dim = 11 # C
elif embedding_type == "one-hot":
  model_dim = vocab_size
input_shape = (model_dim,) # channel size
output_shape = (vocab_size,) # logits
hidden_shape = output_shape # small for now
conveyor_shape = output_shape # these HAVE TO BE THE SAME



rnn_params = init_rnn_params(keys[1], input_shape, hidden_shape, conveyor_shape, output_shape)
if embedding_type == "learned":
  embedding_params = init_embedding_params(keys[0], vocab_size, model_dim)
  rnn_params.append(embedding_params)
jax.device_put(rnn_params, device=gpu_device)



lr = 2e-3
#scheduler = optax.schedules.linear_onecycle_schedule(
#  transition_steps=300000,
#  peak_value=lr,
#  pct_start = 0.3,
#  pct_final = 0.9,
#  div_factor = 25,
#  final_div_factor=100000,
#)
#optimizer = optax.chain(
#  optax.scale_by_adam(),
#  optax.scale_by_schedule(scheduler),
#  optax.scale(-1), # params += -learning_rate x grads
#)

# https://stackoverflow.com/a/53046624
minibatch_size = 500
minibatch_lr_scaling = jnp.sqrt(minibatch_size)
optimizer = optax.adam(learning_rate=lr)
opt_state = optimizer.init(rnn_params)


@jax.jit
def train_step(rnn_params, xtokens, ytokens, opt_state):
  loss, grads = jax.value_and_grad(get_loss)(rnn_params, xtokens, ytokens)
  param_updates, updated_opt_state = optimizer.update(grads, opt_state, rnn_params)
  updated_params = optax.apply_updates(rnn_params, param_updates)
  return loss, updated_params, updated_opt_state


sample_size = 300000 # num of post samples to train on

sequences = jnp.array(sequence_token_sets[:sample_size])
print_every = 1
print(f'Training on {sequences.shape[0]} posts')
last_time = time.time()
#with jax.profiler.trace('./tmp/run'):
for train_sample_idx in range(sample_size - 1):
  # train step on batch
  xtokens_minibatch = sequences[train_sample_idx:train_sample_idx+minibatch_size]
  ytokens_minibatch = sequences[train_sample_idx+1:train_sample_idx+minibatch_size+1]
  debug_log("xtokens_minibatch: ", xtokens_minibatch.shape)
  loss, rnn_params, opt_state = train_step(rnn_params, xtokens_minibatch, ytokens_minibatch, opt_state)
  
  if train_sample_idx % print_every == 0:
    batch_time = time.time() - last_time
    last_time = time.time()
    
    show_output = True
    if show_output:
      logits = forward(rnn_params, embed_tokens(xtokens_minibatch)) # B, T, C
      yhattokens = jnp.argmax(logits, axis=-1) # B, T
      debug_log("logits:", logits.shape)
      yhat_minibatch = jnp.argmax(logits, axis=-1)
      debug_log("yhat_minibatch:", yhat_minibatch.shape)
      prediction = decode(xtokens_minibatch[-1]).replace('\n', '')
      prediction_out = decode(yhat_minibatch[-1]).replace('\n', '')
      print(f'minibatch {train_sample_idx:4.0f}  loss/seq={loss}  samples/s={minibatch_size/batch_time:0.2f}  prediction: "{prediction}" => "{prediction_out}"')
    else:
      print(f'minibatch {train_sample_idx:4.0f}  loss/seq={loss/xtokens_minibatch.shape[0]:3.4f}  tsteps/s={xtokens_minibatch.shape[0]/batch_time:0.2f}')


2024-12-09 13:19:43.870415: W external/xla/xla/tsl/framework/bfc_allocator.cc:497] Allocator (GPU_0_bfc) ran out of memory trying to allocate 15.01MiB (rounded to 15737344)requested by op 
2024-12-09 13:19:43.873072: W external/xla/xla/tsl/framework/bfc_allocator.cc:508] ****************************************************************************************************
E1209 13:19:43.873101  246714 pjrt_stream_executor_client.cc:3085] Execution of replica 0 failed: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 15737220 bytes.


ValueError: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 15737220 bytes.

In [12]:
def inference(xchars):
  xbow = embed_chars(xchars)[None, :] # from T,C to B,T,C where B=1
  logits = forward(rnn_params, xbow) # 1, T, C
  yhatbow = jnp.argmax(logits, axis=-1)[0] #1, T => T
  yhatbow_chars = decode(yhatbow) # T
  return yhatbow_chars # T

text = 'post:'
print(text, end='')
for i in range(100):
  current_input = text[-sequence_length:] # final $seq_length chars
  next_char = inference(current_input)[-1]
  text += next_char
  print(next_char, end='')

post: @ante ante ante ante ante ante ante ante ante ante ante ante ante ante ante ante ante ante ante ant

In [7]:
import jax
import jax.numpy as jnp
with jax.profiler.trace('./tmp/trace'):
  for i in range(1000):
    x = jax.nn.log_softmax(jnp.arange(100000000))
  y = x + 100


2024-12-02 16:14:12.800525: E external/xla/xla/python/profiler/internal/python_hooks.cc:400] Can't import tensorflow.python.profiler.trace
/home/dan/Desktop/ai_gym/.venv/lib/python3.10/site-packages/jax/_src/lax/lax.py:2953: RuntimeWarning: invalid value encountered in cast
  out = np.array(c).astype(eqn.params['new_dtype'])
2024-12-02 16:14:22.047804: E external/xla/xla/python/profiler/internal/python_hooks.cc:400] Can't import tensorflow.python.profiler.trace
